In [9]:
# Reads R1.fastq and R2.fastq files;
# selects reads with proper cell barcode;
# produces a new _cbc.fastq.gz file.
import sys, os
import itertools as it
import argparse as argp
import numpy as np
import gzip
import pandas as pd
from pandas.io.parsers import read_csv
from collections import Counter
import glob

## function to identify cells from barcodes, allowing some edit distances ####

In [10]:
def find_compatible_barcodes(barcode, HDmax = 0):
    """Given a barcode sequence and a maximum Hammin distance, it returns a list of compatible barcode sequences"""
    nt = ['N'] if HDmax == 0 else ['N','C','T','G','A']
    HDmax = 1 if HDmax == 0 else HDmax
    compatible_barcodes = set([barcode])
    for hd in range(1, HDmax+1):
        comb = [''.join(l) for l in it.product(nt, repeat = hd)]
        for c in comb:
            for p in it.permutations(range(len(barcode)), hd):
                s0 = barcode
                for x, l in zip(p, c):
                    s0 = s0[:x] + l + s0[x+1:]
                compatible_barcodes.add(s0)
    return list(compatible_barcodes)

## check input variables ####

In [19]:
## no idea
parser = argp.ArgumentParser(description = 'Concatenates bcread to bioread qname.')
parser.add_argument('--fq1', help = 'Fastq file for read 1')
parser.add_argument('--fq2', help = 'Fastq file for read 2')
parser.add_argument('--fq3', help = 'Fastq file for read 3')
parser.add_argument('--cbcfile', '-cbcf', help = 'cell specific barcode file. Please, provide full name')
parser.add_argument('--cbchd', help = 'collapse cell barcodes with the given hamming distance', type = int, default = 0)
parser.add_argument('--outdir', help = 'output directory for cbc.fastq.gz and log files', type = str, default = './')
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--fq1 FQ1] [--fq2 FQ2] [--fq3 FQ3]
                             [--cbcfile CBCFILE] [--cbchd CBCHD]
                             [--outdir OUTDIR]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/paupascual/.local/share/jupyter/runtime/kernel-ea97847d-8d4a-4c26-bb13-ae8cafe3c365.json


SystemExit: 2

In [15]:
fq1 = '22089_1_AAACGGCG_S1_L001_R1_001.fastq.gz'
fq2 = '22089_1_AAACGGCG_S1_L001_R2_001.fastq.gz'
fq3 = '22089_1_AAACGGCG_S1_L001_R3_001.fastq.gz'
lcbc = 14
lumi = 10
cbcfile = args.cbcfile
hd = args.cbchd
outdir = args.outdir

NameError: name 'args' is not defined

## Find input fastq files ####

In [16]:
if not os.path.isfile(fq1) or not os.path.isfile(fq2) or not os.path.isfile(fq3):
    sys.exit('fastq files not found')

In [17]:
fqr = fq1[: min([i for i, (c1, c2) in enumerate(zip(fq1, fq2)) if c1 != c2])-2]
if '/' in fqr:
    fqr = fqr[:fqr.rindex('/')]

## Read barcodes and expand set according to input hamming distance ####

In [18]:
if not os.path.isfile(cbcfile):
    sys.exit("Barcode file not found")

NameError: name 'cbcfile' is not defined

In [ ]:
bc_df = read_csv(cbcfile, sep = '\t', names = ['bc','cellID'], index_col = 0)
bc_df['compatible_bcs'] = bc_df.apply(lambda x: find_compatible_barcodes(x.name, hd), axis = 1)
cnt_allbcs = Counter([x for idx in bc_df.index for x in bc_df.loc[idx, 'compatible_bcs']])
allbc_df = pd.DataFrame({x: {'cellID': bc_df.loc[idx,'cellID'], 'original': idx} for idx in bc_df.index for x in bc_df.loc[idx, 'compatible_bcs'] if cnt_allbcs[x]==1}).T

# Create output directory if it does not exist ####

In [ ]:
if not os.path.isdir(outdir):
    os.system('mkdir '+outdir)

## Read fastq files and assign cell barcode and UMI ####

In [ ]:
fout = open(outdir + '/' + fqr + '_cbc.fastq', 'w')
ns = 0
with gzip.open(fq1) as f1, gzip.open(fq2) as f2, gzip.open(fq3) as f3: 
    for idx, (l1, l2, l3) in enumerate(zip(f1, f2, f3)):
        l1, l2, l3 = [str(l.rstrip().rsplit()[0], 'utf-8') for l in [l1,l2,l3]]
        l = np.mod(idx,4)
        if l == 0:
            n1, n2, n3 = l1, l2, l3
            if not n1 == n2 == n3:
                print (n1, n2, n3)
                sys.exit('fastq files not syncrhonized (@name)')
        if l == 1:
            s1, s2, s3 = l1, l2, l3
        if l == 2:
            p1, p2, p3 = l1[0], l2[0], l3[0]
            if not p1 == p2 == p3: # == '+':
                print(l1, l2, l3, p1, p2)
                sys.exit('fastq files not synchronized (+)')
        if l == 3:
            q1, q2, q3 = l1, l2, l3
            if len(q1) != len(s1) or len(q2) != len(s2) or len(q3) != len(s3):
                sys.exit('phred and read length not mathch!')
            cellbcseq = s2
            cellbcphred = q2
            s = s1
            q = q1
            
            umiseq = s3
            umiphred = q3
            try:
                cellID, originalBC = allbc_df.loc[cellbcseq]
                ns += 1
                cellbcphred = ''.join([chr(ord(c)+32) for c in cellbcphred])
                umiphred = ''.join([chr(ord(c)+32) for c in umiphred])
                name = ';'.join([n1] + [':'.join(x) for x in zip(['SS','CB','QT','RX','RQ','SM'], [cellbcseq, originalBC, cellbcphred, umiseq, umiphred, str(cellID).zfill(3)])])
                fout.write( '\n'.join([name, s, '+', q, '']))
            except: 
                continue

In [ ]:
nt = (idx+1)/4
fout.close()

## LOG ####

In [ ]:
fout = open(outdir + '/' + fqr + '.log', 'w')
fout.write('=> to generate cbc file <=\n')
fout.write(', '.join(['fastq file:', str(fqr),'\n']))
fout.write(', '.join(['total sequenced reads:', str(nt), '\n']))
fout.write(', '.join(['reads with proper barcodes:', str(ns), str(1.0*ns/nt), '\n']))
fout.close()

## zip fastq file ####

In [ ]:
os.system('gzip '+ outdir + '/' + fqr + '_cbc.fastq')